In [3]:
%pip install tensorflow_datasets
%pip install tensorflow

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
import numpy as np
from sklearn.model_selection import KFold

2024-05-02 20:38:46.675391: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-02 20:38:46.678497: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-02 20:38:46.718149: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-02 20:38:47.655572: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [6]:
import os
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import numpy as np

# Load your image dataset and preprocessing here
# Assume X is your image data and Y is your labels
# Modify this part according to your image dataset loading and preprocessing

# Define your image data (X) and labels (Y) here
# Example: X = ... # Load images using some method (e.g., tf.keras.preprocessing.image_dataset_from_directory)
X = tf.keras.preprocessing.image_dataset_from_directory(
    "dataset/data",
    labels='inferred',
    label_mode='int',
    color_mode='rgb',  # Mengatur mode warna menjadi grayscale
    batch_size=30,
    image_size=(52, 52),
    shuffle=True,
    seed=123,
)
# Example: Y = ... # Load labels (may need to be encoded)
def extract_labels(image, label):
    return label

# Map the dataset to extract labels
Y = X.map(extract_labels)

# Split data into train and test sets
# Convert dataset to numpy arrays
X_list = []
Y_list = []
for images, labels in X.as_numpy_iterator():
    X_list.extend(images)
    Y_list.extend(labels)
X = np.array(X_list)
Y = np.array(Y_list)

# Split data into train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=123)

# Convert labels to one-hot vectors
Y_train = to_categorical(Y_train, num_classes=60)
Y_test = to_categorical(Y_test, num_classes=60)

# Define the ResNet50 model
def build_resnet50_model():
    # Membuat base model ResNet50 tanpa lapisan Fully Connected terakhir
    base_model = ResNet50(
        include_top=False,
        weights='imagenet',
        input_shape=(52, 52, 3),
    )

    # Menghentikan layer-layer yang ada pada base model agar tidak ter-update selama pelatihan
    for layer in base_model.layers:
        layer.trainable = False

    # Membangun model dengan menambahkan lapisan kustom di atas base model
    x = base_model.output
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    predictions = Dense(60, activation='softmax')(x)

    # Membuat model gabungan
    model = Model(inputs=base_model.input, outputs=predictions)

    return model

# Initialize k-Fold
kf = KFold(n_splits=5, shuffle=False, random_state=None)

# Execute K-fold cross-validation
fold_no = 1
acc_per_fold = [] # Save accuracy from each fold

# Train the model for each split (fold)
for train_index, test_index in kf.split(X_train):
    print('Training for fold', fold_no)
    print ('Train Index', train_index)
    print ('Test Index', test_index)

    # Get train and test data for this fold
    X_train_fold, X_val_fold = X_train[train_index], X_train[test_index]
    Y_train_fold, Y_val_fold = Y_train[train_index], Y_train[test_index]

    # Build the model
    model = build_resnet50_model()

    # Compile the model
    model.compile(optimizer=Adam(),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    # Train the model
    history = model.fit(X_train_fold, Y_train_fold, batch_size=30, epochs=100, validation_data=(X_val_fold, Y_val_fold))

    # # Evaluate the model
    # y_pred = model.predict(X_test)
    # y_pred_binary = (y_pred >= 0.5).astype(int)
    # accuracy = accuracy_score(Y_test, y_pred_binary)
    # acc_per_fold.append(accuracy)

    # Evaluate the model
    y_pred = model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    true_classes = np.argmax(Y_test, axis=1)
    accuracy = accuracy_score(true_classes, y_pred_classes)
    acc_per_fold.append(accuracy)

    # Print predicted and true classes
    print("Predicted Classes:", y_pred_classes)
    print("True Classes:", true_classes)
    
    fold_no += 1

# Save the model after all folds are processed
model_save_path = "models/final_model.h5"
model.save(model_save_path)
print(f"Final model saved to {model_save_path}")

# Print accuracy for each fold
for i, acc in enumerate(acc_per_fold, 1):
    print(f'Accuracy for fold {i}: {acc}')

# Print Average Accuracy 
total_acc = sum(acc_per_fold)  # Summing up all accuracies
num_folds = len(acc_per_fold)  # Getting the number of folds

if num_folds > 0:
    average_acc = total_acc / num_folds  # Calculating average accuracy
    print(f'Average Accuracy: {average_acc:.2f}')
else:
    print("No folds to calculate average accuracy.")

Found 240 files belonging to 60 classes.


Training for fold 1
Train Index [ 39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54  55  56
  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72  73  74
  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90  91  92
  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108 109 110
 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128
 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146
 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164
 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182
 183 184 185 186 187 188 189 190 191]
Test Index [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38]


2024-05-02 19:54:57.788406: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 1/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 7s 375ms/step - accuracy: 0.0156 - loss: 0.8269 - val_accuracy: 0.0513 - val_loss: 0.2070
Epoch 2/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 133ms/step - accuracy: 0.0690 - loss: 0.1657 - val_accuracy: 0.1538 - val_loss: 0.2163
Epoch 3/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 152ms/step - accuracy: 0.2315 - loss: 0.1253 - val_accuracy: 0.1538 - val_loss: 0.1833
Epoch 4/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 140ms/step - accuracy: 0.3219 - loss: 0.0998 - val_accuracy: 0.2564 - val_loss: 0.1385
Epoch 5/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 136ms/step - accuracy: 0.5028 - loss: 0.0642 - val_accuracy: 0.3590 - val_loss: 0.1036
Epoch 6/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 133ms/step - accuracy: 0.8357 - loss: 0.0328 - val_accuracy: 0.4872 - val_loss: 0.0846
Epoch 7/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 136ms/step - accuracy: 0.8086 - loss: 0.0318 - val_accuracy: 0.4872 - val_loss: 0.0731
Epoch 8/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 146ms/step - accuracy: 0.9142 - loss: 0.0201 - val_accuracy: 0.5128 - v

Predicted Classes: [30  7 47 58 28  6 14  2 55 52 26 36 11 23 13 22 33 37 50 49 14  8 38 59
  7 19  3  0 48 31 50  3  6 37 17 44 53 38 13 36 24 16 32 35 40  8 30 51]
True Classes: [30  9 47 58 28  6 14  2 55 52 26 36 11 23 13 22 33 12 50 49 14  8 38 59
  9 19  3  0 48 31 50  3  6  9 17 44 53 38 13 36 24 16 32 35 40  8 30 51]
Final model saved to models/final_model.h5
Accuracy for fold 1: 0.9791666666666666
Accuracy for fold 2: 0.8333333333333334
Accuracy for fold 3: 0.8958333333333334
Accuracy for fold 4: 0.875
Accuracy for fold 5: 0.9166666666666666
Average Accuracy: 0.90


TRY ALL DATASET

In [2]:
import os
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import numpy as np

# Load your image dataset and preprocessing here
# Assume X is your image data and Y is your labels
# Modify this part according to your image dataset loading and preprocessing

# Define your image data (X) and labels (Y) here
# Example: X = ... # Load images using some method (e.g., tf.keras.preprocessing.image_dataset_from_directory)
X = tf.keras.preprocessing.image_dataset_from_directory(
    "dataset/dataset_restructure",
    labels='inferred',
    label_mode='int',
    color_mode='rgb',  # Mengatur mode warna menjadi grayscale
    batch_size=30,
    image_size=(52, 52),
    shuffle=True,
    seed=123,
)
# Example: Y = ... # Load labels (may need to be encoded)
def extract_labels(image, label):
    return label

# Map the dataset to extract labels
Y = X.map(extract_labels)

# Split data into train and test sets
# Convert dataset to numpy arrays
X_list = []
Y_list = []
for images, labels in X.as_numpy_iterator():
    X_list.extend(images)
    Y_list.extend(labels)
X = np.array(X_list)
Y = np.array(Y_list)

# Split data into train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=123)

# Convert labels to one-hot vectors
Y_train = to_categorical(Y_train, num_classes=60)
Y_test = to_categorical(Y_test, num_classes=60)

# Define the ResNet50 model
def build_resnet50_model():
    # Membuat base model ResNet50 tanpa lapisan Fully Connected terakhir
    base_model = ResNet50(
        include_top=False,
        weights='imagenet',
        input_shape=(52, 52, 3),
    )

    # Menghentikan layer-layer yang ada pada base model agar tidak ter-update selama pelatihan
    for layer in base_model.layers:
        layer.trainable = False

    # Membangun model dengan menambahkan lapisan kustom di atas base model
    x = base_model.output
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    predictions = Dense(60, activation='softmax')(x)

    # Membuat model gabungan
    model = Model(inputs=base_model.input, outputs=predictions)

    return model

# Initialize k-Fold
kf = KFold(n_splits=5, shuffle=False, random_state=None)

# Execute K-fold cross-validation
fold_no = 1
acc_per_fold = [] # Save accuracy from each fold

# Train the model for each split (fold)
for train_index, test_index in kf.split(X_train):
    print('Training for fold', fold_no)
    print ('Train Index', train_index)
    print ('Test Index', test_index)

    # Get train and test data for this fold
    X_train_fold, X_val_fold = X_train[train_index], X_train[test_index]
    Y_train_fold, Y_val_fold = Y_train[train_index], Y_train[test_index]

    # Build the model
    model = build_resnet50_model()

    # Compile the model
    model.compile(optimizer=Adam(),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    # Train the model
    history = model.fit(X_train_fold, Y_train_fold, batch_size=30, epochs=100, validation_data=(X_val_fold, Y_val_fold))

    # # Evaluate the model
    # y_pred = model.predict(X_test)
    # y_pred_binary = (y_pred >= 0.5).astype(int)
    # accuracy = accuracy_score(Y_test, y_pred_binary)
    # acc_per_fold.append(accuracy)

    # Evaluate the model
    y_pred = model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    true_classes = np.argmax(Y_test, axis=1)
    accuracy = accuracy_score(true_classes, y_pred_classes)
    acc_per_fold.append(accuracy)

    # Print predicted and true classes
    print("Predicted Classes:", y_pred_classes)
    print("True Classes:", true_classes)
    
    fold_no += 1

# Save the model after all folds are processed
model_save_path = "models/final_model.h5"
model.save(model_save_path)
print(f"Final model saved to {model_save_path}")

# Print accuracy for each fold
for i, acc in enumerate(acc_per_fold, 1):
    print(f'Accuracy for fold {i}: {acc}')

# Print Average Accuracy 
total_acc = sum(acc_per_fold)  # Summing up all accuracies
num_folds = len(acc_per_fold)  # Getting the number of folds

if num_folds > 0:
    average_acc = total_acc / num_folds  # Calculating average accuracy
    print(f'Average Accuracy: {average_acc:.2f}')
else:
    print("No folds to calculate average accuracy.")

Found 960 files belonging to 60 classes.


2024-05-02 20:38:57.770050: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Training for fold 1
Train Index [154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171
 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189
 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207
 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225
 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243
 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261
 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 278 279
 280 281 282 283 284 285 286 287 288 289 290 291 292 293 294 295 296 297
 298 299 300 301 302 303 304 305 306 307 308 309 310 311 312 313 314 315
 316 317 318 319 320 321 322 323 324 325 326 327 328 329 330 331 332 333
 334 335 336 337 338 339 340 341 342 343 344 345 346 347 348 349 350 351
 352 353 354 355 356 357 358 359 360 361 362 363 364 365 366 367 368 369
 370 371 372 373 374 375 376 377 378 379 380 381 382 383 384 385 386 387
 388 389 390 391 39

Predicted Classes: [49 16 13  2  2 47 37  5 40 12 29 13 37 12  1 31  0 26  2 35 11 22 58 42
 26 54 14 21 21 23 31  8 48 42 51 45 19  0 43  3 29  2 16 56 47 49 33 55
  9 58  3 15 11 42 36  8 59  1 38  1 54 55 35 30 54 52 21  3 19 44 16 13
 25  9 41 27 59 16 32 34 57 37 15 17  1 56 10 20 34 52 48 21  9 15 43  4
  9 21 16 46 37 42 13 42 26 44 20  4 52  8 53 38 56 57 37  7 51  8 15 14
 31 53 26 58 16 26  8  5 28 21 48 51 52 22 41 54 38 39 17 41 35 11 21 44
 18 33 34 20 36 27 58 17 59 12 40 33 54 40 12 15 27 16 29 38 56 14 28 29
 59 59  3  4 24 16 23 28 33 31  9 11 45  2 18 30 25 50 20 19 34 25 17 25]
True Classes: [49 16 13  2  2 47 37  5 40 12 29 13 37 12  1 31  0 26  2 35 11 22 58 42
 26 54 14 21 21 23 31  8 48 42 51 45 19  0 43  3 29  2 16 56 47 49 33 55
  9 58  3 15 11 42 36  8 59  1 38  1 54 55 35 30 54 52 21  3 19 44 16 13
 25  9 41 27 59 16 32 34 57 37 15 17  1 56 10 20 34 52 48 21  9 15 43  4
  9 21 16 46 37 42 13 42 26 44 20  4 52  8 53 38 56 57 37  7 51  8 15 14
 31 53 26 58 16 2

TRY ALL DATASET AND UPDATE VALUE SHUFFLE TO TRUE

In [3]:
import os
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import numpy as np

# Load your image dataset and preprocessing here
# Assume X is your image data and Y is your labels
# Modify this part according to your image dataset loading and preprocessing

# Define your image data (X) and labels (Y) here
# Example: X = ... # Load images using some method (e.g., tf.keras.preprocessing.image_dataset_from_directory)
X = tf.keras.preprocessing.image_dataset_from_directory(
    "dataset/dataset_restructure",
    labels='inferred',
    label_mode='int',
    color_mode='rgb',  # Mengatur mode warna menjadi grayscale
    batch_size=30,
    image_size=(52, 52),
    shuffle=True,
    seed=123,
)
# Example: Y = ... # Load labels (may need to be encoded)
def extract_labels(image, label):
    return label

# Map the dataset to extract labels
Y = X.map(extract_labels)

# Split data into train and test sets
# Convert dataset to numpy arrays
X_list = []
Y_list = []
for images, labels in X.as_numpy_iterator():
    X_list.extend(images)
    Y_list.extend(labels)
X = np.array(X_list)
Y = np.array(Y_list)

# Split data into train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Convert labels to one-hot vectors
Y_train = to_categorical(Y_train, num_classes=60)
Y_test = to_categorical(Y_test, num_classes=60)

# Define the ResNet50 model
def build_resnet50_model():
    # Membuat base model ResNet50 tanpa lapisan Fully Connected terakhir
    base_model = ResNet50(
        include_top=False,
        weights='imagenet',
        input_shape=(52, 52, 3),
    )

    # Menghentikan layer-layer yang ada pada base model agar tidak ter-update selama pelatihan
    for layer in base_model.layers:
        layer.trainable = False

    # Membangun model dengan menambahkan lapisan kustom di atas base model
    x = base_model.output
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    predictions = Dense(60, activation='softmax')(x)

    # Membuat model gabungan
    model = Model(inputs=base_model.input, outputs=predictions)

    return model

# Initialize k-Fold
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Execute K-fold cross-validation
fold_no = 1
acc_per_fold = [] # Save accuracy from each fold

# Train the model for each split (fold)
for train_index, test_index in kf.split(X_train):
    print('Training for fold', fold_no)
    print ('Train Index', train_index)
    print ('Test Index', test_index)

    # Get train and test data for this fold
    X_train_fold, X_val_fold = X_train[train_index], X_train[test_index]
    Y_train_fold, Y_val_fold = Y_train[train_index], Y_train[test_index]

    # Build the model
    model = build_resnet50_model()

    # Compile the model
    model.compile(optimizer=Adam(),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    # Train the model
    history = model.fit(X_train_fold, Y_train_fold, batch_size=30, epochs=100, validation_data=(X_val_fold, Y_val_fold))

    # # Evaluate the model
    # y_pred = model.predict(X_test)
    # y_pred_binary = (y_pred >= 0.5).astype(int)
    # accuracy = accuracy_score(Y_test, y_pred_binary)
    # acc_per_fold.append(accuracy)

    # Evaluate the model
    y_pred = model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    true_classes = np.argmax(Y_test, axis=1)
    accuracy = accuracy_score(true_classes, y_pred_classes)
    acc_per_fold.append(accuracy)

    # Print predicted and true classes
    print("Predicted Classes:", y_pred_classes)
    print("True Classes:", true_classes)
    
    fold_no += 1

# Save the model after all folds are processed
model_save_path = "models/final_model.h5"
model.save(model_save_path)
print(f"Final model saved to {model_save_path}")

# Print accuracy for each fold
for i, acc in enumerate(acc_per_fold, 1):
    print(f'Accuracy for fold {i}: {acc}')

# Print Average Accuracy 
total_acc = sum(acc_per_fold)  # Summing up all accuracies
num_folds = len(acc_per_fold)  # Getting the number of folds

if num_folds > 0:
    average_acc = total_acc / num_folds  # Calculating average accuracy
    print(f'Average Accuracy: {average_acc:.2f}')
else:
    print("No folds to calculate average accuracy.")

Found 960 files belonging to 60 classes.


2024-05-02 21:15:09.605157: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Training for fold 1
Train Index [  0   1   3   4   5   6   8   9  11  12  13  14  15  16  17  18  19  20
  21  22  24  25  26  27  28  29  32  34  35  36  37  38  40  41  42  43
  44  45  46  47  48  50  51  52  53  56  57  58  59  60  61  62  64  68
  69  70  71  73  74  75  80  82  83  85  87  88  89  90  91  92  93  94
  95  96  98  99 100 102 103 104 105 106 107 108 110 111 112 113 114 115
 116 117 119 121 122 123 124 125 126 127 128 129 130 131 132 134 135 136
 138 139 140 141 142 143 144 145 146 147 149 150 151 152 153 154 156 157
 158 159 160 161 162 163 164 165 166 167 169 170 171 172 173 175 176 177
 178 179 180 182 183 184 185 186 187 188 189 190 191 194 195 197 198 200
 201 202 203 205 206 207 214 216 217 219 221 222 223 224 225 226 228 229
 230 231 232 233 235 236 237 238 240 241 242 243 245 246 247 248 249 251
 252 253 255 256 257 258 260 261 262 263 264 266 267 268 269 270 271 272
 273 274 276 277 278 279 280 282 283 284 285 286 287 288 289 291 292 293
 294 295 297 298 30

Predicted Classes: [44 37 24 36  8 22 31 46 19 13 39 47 33 47 40 11 49 49 25 46 23 55  1 54
 15 58 57 50 56 21 28 39 53  5 55 20  0 10  0 32  7  6 54 57 51 20 36 45
 15 50 22 42  0 30  6 13  2 28 57 52 34 30 52 23 32 13 19 44 59 44 55 22
 34 37  0 33 52 35  3  0 42  9 18 41 58 30 16 18  7 38 21 24 24 17 56 40
 35 25 41 44 32  8  9 11 32 21 43  4 36 58  5 50 52 27  4 20 42 57 27 40
 40 17 37  8 48 47 21  3 28 53  9 38 55 47 20 43  8 10  6 26 54 46 40  0
  0 36 32  4 51 46 23 57 37  8  8 42 37 37 50 28  0 30  1 20  0 21 41 18
 55 13 41 58 19 29 48 57 26 35  1 31 34 34 27 47 54 31 18 50 14 53 18 58]
True Classes: [44 37 24 36  8 22 31 46 19 13 39 47 33 47 40 11 49 49 25 46 23 55  1 54
 15 58 57 50 56 21 28 39 53  5 55 20  0 10  0 32  7  6 54 57 51 20 36 45
 15 50 22 42  0 30  6 13  2 28 57 52 34 30 52 23 32 13 19 44 59 44 55 22
 34 37  0 33 52 35  3  0 42  9 18 41 58 30 16 18  7 38 21 24 24 17 56 40
 35 25 41 44 32  8  9 11 32 21 43  4 36 58  5 50 52 27  4 20 42 57 27 40
 40 17 37  8 48 4

TRY ALL DATASET
SHUFFLE = TRUE, EPOCH = 10, BATCH SIZE = 30

In [4]:
import os
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import numpy as np

# Load your image dataset and preprocessing here
# Assume X is your image data and Y is your labels
# Modify this part according to your image dataset loading and preprocessing

# Define your image data (X) and labels (Y) here
# Example: X = ... # Load images using some method (e.g., tf.keras.preprocessing.image_dataset_from_directory)
X = tf.keras.preprocessing.image_dataset_from_directory(
    "dataset/dataset_restructure",
    labels='inferred',
    label_mode='int',
    color_mode='rgb',  # Mengatur mode warna menjadi grayscale
    batch_size=30,
    image_size=(52, 52),
    shuffle=True,
    seed=123,
)
# Example: Y = ... # Load labels (may need to be encoded)
def extract_labels(image, label):
    return label

# Map the dataset to extract labels
Y = X.map(extract_labels)

# Split data into train and test sets
# Convert dataset to numpy arrays
X_list = []
Y_list = []
for images, labels in X.as_numpy_iterator():
    X_list.extend(images)
    Y_list.extend(labels)
X = np.array(X_list)
Y = np.array(Y_list)

# Split data into train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Convert labels to one-hot vectors
Y_train = to_categorical(Y_train, num_classes=60)
Y_test = to_categorical(Y_test, num_classes=60)

# Define the ResNet50 model
def build_resnet50_model():
    # Membuat base model ResNet50 tanpa lapisan Fully Connected terakhir
    base_model = ResNet50(
        include_top=False,
        weights='imagenet',
        input_shape=(52, 52, 3),
    )

    # Menghentikan layer-layer yang ada pada base model agar tidak ter-update selama pelatihan
    for layer in base_model.layers:
        layer.trainable = False

    # Membangun model dengan menambahkan lapisan kustom di atas base model
    x = base_model.output
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    predictions = Dense(60, activation='softmax')(x)

    # Membuat model gabungan
    model = Model(inputs=base_model.input, outputs=predictions)

    return model

# Initialize k-Fold
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Execute K-fold cross-validation
fold_no = 1
acc_per_fold = [] # Save accuracy from each fold

# Train the model for each split (fold)
for train_index, test_index in kf.split(X_train):
    print('Training for fold', fold_no)
    print ('Train Index', train_index)
    print ('Test Index', test_index)

    # Get train and test data for this fold
    X_train_fold, X_val_fold = X_train[train_index], X_train[test_index]
    Y_train_fold, Y_val_fold = Y_train[train_index], Y_train[test_index]

    # Build the model
    model = build_resnet50_model()

    # Compile the model
    model.compile(optimizer=Adam(),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    # Train the model
    history = model.fit(X_train_fold, Y_train_fold, batch_size=30, epochs=10, validation_data=(X_val_fold, Y_val_fold))

    # # Evaluate the model
    # y_pred = model.predict(X_test)
    # y_pred_binary = (y_pred >= 0.5).astype(int)
    # accuracy = accuracy_score(Y_test, y_pred_binary)
    # acc_per_fold.append(accuracy)

    # Evaluate the model
    y_pred = model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    true_classes = np.argmax(Y_test, axis=1)
    accuracy = accuracy_score(true_classes, y_pred_classes)
    acc_per_fold.append(accuracy)

    # Print predicted and true classes
    print("Predicted Classes:", y_pred_classes)
    print("True Classes:", true_classes)
    
    fold_no += 1

# Save the model after all folds are processed
model_save_path = "models/final_model.h5"
model.save(model_save_path)
print(f"Final model saved to {model_save_path}")

# Print accuracy for each fold
for i, acc in enumerate(acc_per_fold, 1):
    print(f'Accuracy for fold {i}: {acc}')

# Print Average Accuracy 
total_acc = sum(acc_per_fold)  # Summing up all accuracies
num_folds = len(acc_per_fold)  # Getting the number of folds

if num_folds > 0:
    average_acc = total_acc / num_folds  # Calculating average accuracy
    print(f'Average Accuracy: {average_acc:.2f}')
else:
    print("No folds to calculate average accuracy.")

Found 960 files belonging to 60 classes.


2024-05-02 22:01:39.482543: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Training for fold 1
Train Index [  0   1   3   4   5   6   8   9  11  12  13  14  15  16  17  18  19  20
  21  22  24  25  26  27  28  29  32  34  35  36  37  38  40  41  42  43
  44  45  46  47  48  50  51  52  53  56  57  58  59  60  61  62  64  68
  69  70  71  73  74  75  80  82  83  85  87  88  89  90  91  92  93  94
  95  96  98  99 100 102 103 104 105 106 107 108 110 111 112 113 114 115
 116 117 119 121 122 123 124 125 126 127 128 129 130 131 132 134 135 136
 138 139 140 141 142 143 144 145 146 147 149 150 151 152 153 154 156 157
 158 159 160 161 162 163 164 165 166 167 169 170 171 172 173 175 176 177
 178 179 180 182 183 184 185 186 187 188 189 190 191 194 195 197 198 200
 201 202 203 205 206 207 214 216 217 219 221 222 223 224 225 226 228 229
 230 231 232 233 235 236 237 238 240 241 242 243 245 246 247 248 249 251
 252 253 255 256 257 258 260 261 262 263 264 266 267 268 269 270 271 272
 273 274 276 277 278 279 280 282 283 284 285 286 287 288 289 291 292 293
 294 295 297 298 30

Predicted Classes: [44 37 24 36  8 22 31 46 19 13 39 47 33 47 40 11 49 49 25 46 23 55  1 54
 15 58 57 50 56 21 28 39 53  5 55 20  0 10  0 32  7  6 54 57 51 20 36 45
 15 50 22 42  0 30  6 13  2 28 57 52 34 30 52 23 32 13 19 44 59 44 55 22
 34 37  0 33 52 35  3  0 42  9 18 41 58 30 16 18  7 38 21 24 24 17 56 40
 35 25 41 44 32  8  9 11 32 21 43  4 36 58  5 50 52 27  4 20 42 57 27 40
 40 17 37  8 48 47 21  3 28 53  9 38 55 47 20 43  8 10  6 26 54 46 40  0
  0 36 32  4 51 46 23 57 37  8  8 42 37 37 50 28  0 30  1 20  0 21 41 18
 55 13 41 58 19 29 48 57 26 35  1 31 34 34 27 47 54 31 18 50 14 53 18 58]
True Classes: [44 37 24 36  8 22 31 46 19 13 39 47 33 47 40 11 49 49 25 46 23 55  1 54
 15 58 57 50 56 21 28 39 53  5 55 20  0 10  0 32  7  6 54 57 51 20 36 45
 15 50 22 42  0 30  6 13  2 28 57 52 34 30 52 23 32 13 19 44 59 44 55 22
 34 37  0 33 52 35  3  0 42  9 18 41 58 30 16 18  7 38 21 24 24 17 56 40
 35 25 41 44 32  8  9 11 32 21 43  4 36 58  5 50 52 27  4 20 42 57 27 40
 40 17 37  8 48 4

BATCH SIZE 32, EPOCH 10

In [5]:
import os
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import numpy as np

# Load your image dataset and preprocessing here
# Assume X is your image data and Y is your labels
# Modify this part according to your image dataset loading and preprocessing

# Define your image data (X) and labels (Y) here
# Example: X = ... # Load images using some method (e.g., tf.keras.preprocessing.image_dataset_from_directory)
X = tf.keras.preprocessing.image_dataset_from_directory(
    "dataset/dataset_restructure",
    labels='inferred',
    label_mode='int',
    color_mode='rgb',  # Mengatur mode warna menjadi grayscale
    image_size=(52, 52),
    shuffle=True,
    seed=123,
)
# Example: Y = ... # Load labels (may need to be encoded)
def extract_labels(image, label):
    return label

# Map the dataset to extract labels
Y = X.map(extract_labels)

# Split data into train and test sets
# Convert dataset to numpy arrays
X_list = []
Y_list = []
for images, labels in X.as_numpy_iterator():
    X_list.extend(images)
    Y_list.extend(labels)
X = np.array(X_list)
Y = np.array(Y_list)

# Split data into train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Convert labels to one-hot vectors
Y_train = to_categorical(Y_train, num_classes=60)
Y_test = to_categorical(Y_test, num_classes=60)

# Define the ResNet50 model
def build_resnet50_model():
    # Membuat base model ResNet50 tanpa lapisan Fully Connected terakhir
    base_model = ResNet50(
        include_top=False,
        weights='imagenet',
        input_shape=(52, 52, 3),
    )

    # Menghentikan layer-layer yang ada pada base model agar tidak ter-update selama pelatihan
    for layer in base_model.layers:
        layer.trainable = False

    # Membangun model dengan menambahkan lapisan kustom di atas base model
    x = base_model.output
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    predictions = Dense(60, activation='softmax')(x)

    # Membuat model gabungan
    model = Model(inputs=base_model.input, outputs=predictions)

    return model

# Initialize k-Fold
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Execute K-fold cross-validation
fold_no = 1
acc_per_fold = [] # Save accuracy from each fold

# Train the model for each split (fold)
for train_index, test_index in kf.split(X_train):
    print('Training for fold', fold_no)
    print ('Train Index', train_index)
    print ('Test Index', test_index)

    # Get train and test data for this fold
    X_train_fold, X_val_fold = X_train[train_index], X_train[test_index]
    Y_train_fold, Y_val_fold = Y_train[train_index], Y_train[test_index]

    # Build the model
    model = build_resnet50_model()

    # Compile the model
    model.compile(optimizer=Adam(),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    # Train the model
    history = model.fit(X_train_fold, Y_train_fold, batch_size=32, epochs=10, validation_data=(X_val_fold, Y_val_fold))

    # # Evaluate the model
    # y_pred = model.predict(X_test)
    # y_pred_binary = (y_pred >= 0.5).astype(int)
    # accuracy = accuracy_score(Y_test, y_pred_binary)
    # acc_per_fold.append(accuracy)

    # Evaluate the model
    y_pred = model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    true_classes = np.argmax(Y_test, axis=1)
    accuracy = accuracy_score(true_classes, y_pred_classes)
    acc_per_fold.append(accuracy)

    # Print predicted and true classes
    print("Predicted Classes:", y_pred_classes)
    print("True Classes:", true_classes)
    
    fold_no += 1

# Save the model after all folds are processed
model_save_path = "models/final_model.h5"
model.save(model_save_path)
print(f"Final model saved to {model_save_path}")

# Print accuracy for each fold
for i, acc in enumerate(acc_per_fold, 1):
    print(f'Accuracy for fold {i}: {acc}')

# Print Average Accuracy 
total_acc = sum(acc_per_fold)  # Summing up all accuracies
num_folds = len(acc_per_fold)  # Getting the number of folds

if num_folds > 0:
    average_acc = total_acc / num_folds  # Calculating average accuracy
    print(f'Average Accuracy: {average_acc:.2f}')
else:
    print("No folds to calculate average accuracy.")

Found 960 files belonging to 60 classes.


2024-05-02 22:12:02.508390: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Training for fold 1
Train Index [  0   1   3   4   5   6   8   9  11  12  13  14  15  16  17  18  19  20
  21  22  24  25  26  27  28  29  32  34  35  36  37  38  40  41  42  43
  44  45  46  47  48  50  51  52  53  56  57  58  59  60  61  62  64  68
  69  70  71  73  74  75  80  82  83  85  87  88  89  90  91  92  93  94
  95  96  98  99 100 102 103 104 105 106 107 108 110 111 112 113 114 115
 116 117 119 121 122 123 124 125 126 127 128 129 130 131 132 134 135 136
 138 139 140 141 142 143 144 145 146 147 149 150 151 152 153 154 156 157
 158 159 160 161 162 163 164 165 166 167 169 170 171 172 173 175 176 177
 178 179 180 182 183 184 185 186 187 188 189 190 191 194 195 197 198 200
 201 202 203 205 206 207 214 216 217 219 221 222 223 224 225 226 228 229
 230 231 232 233 235 236 237 238 240 241 242 243 245 246 247 248 249 251
 252 253 255 256 257 258 260 261 262 263 264 266 267 268 269 270 271 272
 273 274 276 277 278 279 280 282 283 284 285 286 287 288 289 291 292 293
 294 295 297 298 30

Predicted Classes: [40 28  3 38 58 20  4 51 56 13 17 28 48 14 47 55 51 45 52 46 23 36  1 24
 59 55 57 12 36  2 35  0 59 41 56 22  5 11 14 57 43 44 32  5 21 40 12 42
 46 46 59  5 45 55 31 34 38 16 33 17 12 35 19 23 45 29 37 57 23 52 48  5
 48 14 38 24  6 32 37 24 26  9 21 19 13  3 38 35  6 25  4 24 42 15 28  4
 35 25  0 44 57  3  9  2 32  1 44 51  1 58 46 43 27  1  7 41 58 14 32 40
  5 56 19  6 48 28 21 15 52 41  3 18 37 13  0 43 46 16  6  6 25 44 30  3
  0 27 58  5 17 39 53 57 13 32 18 42 37 38 50 25 32 57 18 43 40 16 38 53
 55 40 46 51 56  1 48  7 44  0 21 13 40 39 27 47 11 33 38 54 37 23 18  0]
True Classes: [40 28  3 38 58 20 46 51 56 13 17 28 48 14 47 55 51 45 52 46 23 36  1 24
 59 55 57 12 36  2 35  0 59 41 56 22  5 11 14 57 43 44 32  5 21 40 12 42
 46 46 59  5 45 55 31 34 38 16 33 17 12 35 19 23 45 29 37 57 23 52 48  5
 48 14 38 24  6 32 37 24 26  9 21 19 13  3 38 35  6 25  4 24 42 15 28  4
 35 25  0 44 57  3  9  2 32  1 44 51  1 58 46 43 27  1  7 41 58 14 32 40
  5 56 19  6 48 2

BATCH SIZE = 32, EPOCH = 100

In [6]:
import os
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import numpy as np

# Load your image dataset and preprocessing here
# Assume X is your image data and Y is your labels
# Modify this part according to your image dataset loading and preprocessing

# Define your image data (X) and labels (Y) here
# Example: X = ... # Load images using some method (e.g., tf.keras.preprocessing.image_dataset_from_directory)
X = tf.keras.preprocessing.image_dataset_from_directory(
    "dataset/dataset_restructure",
    labels='inferred',
    label_mode='int',
    color_mode='rgb',  # Mengatur mode warna menjadi grayscale
    image_size=(52, 52),
    shuffle=True,
    seed=123,
)
# Example: Y = ... # Load labels (may need to be encoded)
def extract_labels(image, label):
    return label

# Map the dataset to extract labels
Y = X.map(extract_labels)

# Split data into train and test sets
# Convert dataset to numpy arrays
X_list = []
Y_list = []
for images, labels in X.as_numpy_iterator():
    X_list.extend(images)
    Y_list.extend(labels)
X = np.array(X_list)
Y = np.array(Y_list)

# Split data into train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Convert labels to one-hot vectors
Y_train = to_categorical(Y_train, num_classes=60)
Y_test = to_categorical(Y_test, num_classes=60)

# Define the ResNet50 model
def build_resnet50_model():
    # Membuat base model ResNet50 tanpa lapisan Fully Connected terakhir
    base_model = ResNet50(
        include_top=False,
        weights='imagenet',
        input_shape=(52, 52, 3),
    )

    # Menghentikan layer-layer yang ada pada base model agar tidak ter-update selama pelatihan
    for layer in base_model.layers:
        layer.trainable = False

    # Membangun model dengan menambahkan lapisan kustom di atas base model
    x = base_model.output
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    predictions = Dense(60, activation='softmax')(x)

    # Membuat model gabungan
    model = Model(inputs=base_model.input, outputs=predictions)

    return model

# Initialize k-Fold
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Execute K-fold cross-validation
fold_no = 1
acc_per_fold = [] # Save accuracy from each fold

# Train the model for each split (fold)
for train_index, test_index in kf.split(X_train):
    print('Training for fold', fold_no)
    print ('Train Index', train_index)
    print ('Test Index', test_index)

    # Get train and test data for this fold
    X_train_fold, X_val_fold = X_train[train_index], X_train[test_index]
    Y_train_fold, Y_val_fold = Y_train[train_index], Y_train[test_index]

    # Build the model
    model = build_resnet50_model()

    # Compile the model
    model.compile(optimizer=Adam(),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    # Train the model
    history = model.fit(X_train_fold, Y_train_fold, batch_size=32, epochs=100, validation_data=(X_val_fold, Y_val_fold))

    # # Evaluate the model
    # y_pred = model.predict(X_test)
    # y_pred_binary = (y_pred >= 0.5).astype(int)
    # accuracy = accuracy_score(Y_test, y_pred_binary)
    # acc_per_fold.append(accuracy)

    # Evaluate the model
    y_pred = model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    true_classes = np.argmax(Y_test, axis=1)
    accuracy = accuracy_score(true_classes, y_pred_classes)
    acc_per_fold.append(accuracy)

    # Print predicted and true classes
    print("Predicted Classes:", y_pred_classes)
    print("True Classes:", true_classes)
    
    fold_no += 1

# Save the model after all folds are processed
model_save_path = "models/final_model.h5"
model.save(model_save_path)
print(f"Final model saved to {model_save_path}")

# Print accuracy for each fold
for i, acc in enumerate(acc_per_fold, 1):
    print(f'Accuracy for fold {i}: {acc}')

# Print Average Accuracy 
total_acc = sum(acc_per_fold)  # Summing up all accuracies
num_folds = len(acc_per_fold)  # Getting the number of folds

if num_folds > 0:
    average_acc = total_acc / num_folds  # Calculating average accuracy
    print(f'Average Accuracy: {average_acc:.2f}')
else:
    print("No folds to calculate average accuracy.")

Found 960 files belonging to 60 classes.


2024-05-02 22:19:33.797171: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Training for fold 1
Train Index [  0   1   3   4   5   6   8   9  11  12  13  14  15  16  17  18  19  20
  21  22  24  25  26  27  28  29  32  34  35  36  37  38  40  41  42  43
  44  45  46  47  48  50  51  52  53  56  57  58  59  60  61  62  64  68
  69  70  71  73  74  75  80  82  83  85  87  88  89  90  91  92  93  94
  95  96  98  99 100 102 103 104 105 106 107 108 110 111 112 113 114 115
 116 117 119 121 122 123 124 125 126 127 128 129 130 131 132 134 135 136
 138 139 140 141 142 143 144 145 146 147 149 150 151 152 153 154 156 157
 158 159 160 161 162 163 164 165 166 167 169 170 171 172 173 175 176 177
 178 179 180 182 183 184 185 186 187 188 189 190 191 194 195 197 198 200
 201 202 203 205 206 207 214 216 217 219 221 222 223 224 225 226 228 229
 230 231 232 233 235 236 237 238 240 241 242 243 245 246 247 248 249 251
 252 253 255 256 257 258 260 261 262 263 264 266 267 268 269 270 271 272
 273 274 276 277 278 279 280 282 283 284 285 286 287 288 289 291 292 293
 294 295 297 298 30

Predicted Classes: [40 28  3 38 58 20 55 51 56 13 17 28 48 14 47 55 51 45 52 46 23 36  1 24
 59 55 57 12 36  2 35  0 59 41 56 22  5 11 14 57 43 44 32  5 21 40 12 42
 46 46 59  5 45 55 31 34 38 16 33 17 12 35 19 23 45 29 37 57 23 52 48  5
 48 14 38 24  6 32 37 24 26  9 21 19 13  3 38 35  6 25  4 24 42 15 28  4
 35 25  0 44 57  3  9  2 32  1 44 51  1 58 46 43 27  1  7 41 58 14 32 40
  5 56 19  6 48 28 21 15 52 41  3 18 37 13  0 43 46 16  6  6 25 44 30  3
  0 27 58  5 17 39 53 57 13 32 18 42 37 38 50 25 32 57 18 43 40 16 38 53
 55 40 46 51 56  1 48  7 44  0 21 13 40 39 27 47 11 33 38 54 37 23 18  0]
True Classes: [40 28  3 38 58 20 46 51 56 13 17 28 48 14 47 55 51 45 52 46 23 36  1 24
 59 55 57 12 36  2 35  0 59 41 56 22  5 11 14 57 43 44 32  5 21 40 12 42
 46 46 59  5 45 55 31 34 38 16 33 17 12 35 19 23 45 29 37 57 23 52 48  5
 48 14 38 24  6 32 37 24 26  9 21 19 13  3 38 35  6 25  4 24 42 15 28  4
 35 25  0 44 57  3  9  2 32  1 44 51  1 58 46 43 27  1  7 41 58 14 32 40
  5 56 19  6 48 2